<a href="https://colab.research.google.com/github/eDoggo3779/Water_Insecurity/blob/main/Water_Insecurity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To what extent are plumbing and water insecurity outcomes across U.S. counties associated with education levels and racial/ethnic composition?

In [87]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
df22 = pd.read_csv("/content/drive/MyDrive/DSRP/water_insecurity_2022.csv")
df22.dropna(inplace=True)

df22[['county', 'state']] = df22['name'].str.split(',', n=1, expand=True)
df22['county'] = df22['county'].str.strip()
df22['state'] = df22['state'].str.strip()

df22['plp_scaled'] = df22['percent_lacking_plumbing'].apply(lambda x: np.log10(x) if x > 0 else 0)

df22 = df22[['geoid', 'year', 'name', 'geometry', 'state', 'county', 'total_pop', 'plumbing', 'percent_lacking_plumbing', "plp_scaled"]]

In [89]:
# Steven Manson, Jonathan Schroeder, David Van Riper,
# Katherine Knowles, Tracy Kugler, Finn Roberts, and Steven Ruggles.
# IPUMS National Historical Geographic Information System: Version 19.0 [dataset].
# Minneapolis, MN: IPUMS. 2024. http://doi.org/10.18128/D050.V19.0

edu_df = pd.read_csv("/content/drive/MyDrive/DSRP/nhgis0001_ds261_2022_county - filtered.csv")
edu_df.drop(columns='Unnamed: 5', inplace=True)
edu_df.dropna(inplace=True)

In [90]:
# Steven Manson, Jonathan Schroeder, David Van Riper,
# Katherine Knowles, Tracy Kugler, Finn Roberts, and Steven Ruggles.
# IPUMS National Historical Geographic Information System: Version 19.0 [dataset].
# Minneapolis, MN: IPUMS. 2024. http://doi.org/10.18128/D050.V19.0

race_df = pd.read_csv("/content/drive/MyDrive/DSRP/nhgis0003_ds261_2022_county - filtered.csv")
race_df.dropna(inplace=True)

In [102]:
merged_df = df22.merge(edu_df, on=['geoid', 'year', 'name', 'state', 'county'], how='inner').merge(race_df, on=['geoid', 'year', 'name', 'state', 'county'], how='inner')
merged_df.rename(columns={'total_surveyed_x': 'total_edu', 'total_surveyed_y': 'total_race'}, inplace=True)

In [ ]:
# NOTE THAT THE MAP DOESN'T MAP PUERTO RICO

import plotly.express as px
import json

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Plot
fig = px.choropleth(df22, geojson=counties, locations='geoid', color='plp_scaled', color_continuous_scale='Viridis', hover_name='name', scope='usa')
fig.show()